In [1]:
%load_ext autoreload
%autoreload 2

import jCMIP as jc
import os

In [2]:
# historical:
cmips = ['6']
EXPs  = ['historical']
styr  = 1970
fnyr  = 1999

In [3]:
# ScenarioMIP:
cmips = ['6']
EXPs  = ['ssp126','ssp245','ssp585']
styr  = 2070
fnyr  = 2099

In [4]:
vars  = ['zos']
vtype = 'Omon'

# Find all models with valid data make make slurm file:
for cmip in cmips:
    Clist = jc.readList(('../CMIP' + cmip + 'list'))
    Models = list(Clist.keys())
    for mm in Models:
        Model = Clist[mm]
        print(Model.name)
        if ((Model.Ogrid != 'Unstructured')):                
            for var in vars:
                for EXP in EXPs:
                    ENSs = Model.getENSs(var=var,EXP=EXP,vtype=vtype,gtype='*')
                    for ENS in ENSs:
                        gtype = 'gn'
                        # Check if files needed for computation exist:
                        Files = Model.getFiles(var=var,EXP=EXP,ENS=ENS,vtype=vtype,gtype=gtype)
                        if ((len(Files) == 0) & (cmip == '6')):
                            gtype = 'gr'
                            Files = Model.getFiles(var=var,EXP=EXP,ENS=ENS,vtype=vtype,gtype=gtype)
                            if ((len(Files) == 0) & (cmip == '6')):
                                gtype = 'gr1'
                                Files = Model.getFiles(var=var,EXP=EXP,ENS=ENS,vtype=vtype,gtype=gtype)

                        if len(Files) != 0:
                            # Check if computation has already been completed:
                            outfile = ('/home/users/jmecking001/DATADIR/ACSIS/CMIP' + cmip + '/' + Model.name + '/' + EXP + '/' + ENS + '/' + var + '_' + Model.name + '_' + EXP + '_'
                                           + ENS + '_' + gtype + '_' + str(styr) + '01-' + str(fnyr) + '12_SC.nc')
                            
                            if not os.path.isfile(outfile):
                                Files.sort()
                                stf = int(Files[0].split('_')[-1].split('-')[0][:4])
                                fnf = int(Files[-1].split('_')[-1].split('-')[1][:4])
                                if ((stf <= styr) & (fnf >= fnyr)):
                                    print('computing: ' + outfile)
                                    # Generate submit script for lotus:
                                    lfile = ('slurm_files/SC_mean_' + var + '_' + Model.name + '_' +  EXP + '_' + ENS + '.slurm')

                                    line1 = '#!/bin/bash \n'
                                    line2 = '#SBATCH --account=short4hr \n'
                                    line3 = '#SBATCH -p short-serial-4hr \n'
                                    line4 = '#SBATCH -o %J.out \n'
                                    line5 = '#SBATCH -e %J.err \n'
                                    line6 = '#SBATCH --time 4:00:00 \n'
                                    line7 = '\n'
                                    line8 = 'conda activate working \n'
                                    line9 = ('python ../compute_means_SC.py ' + cmip + ' ' + Model.name + ' ' + EXP + ' ' + ENS + ' '
                                    + var + ' ' + vtype + ' ' + str(styr) + ' ' + str(fnyr) + ' ' + outfile + ' ' + gtype + ' ../' + lfile)

                                    # Out of memory models (second pass):
                                    #line2 = '#SBATCH -p high-mem \n'
                                    #line3 = '#SBATCH --mem=32000 \n'

                                    file = open(lfile,'w') 
                                    file.writelines([line1, line2, line3, line4, line5, line6, line7, line8, line9])

                                    file.close()



ACCESS-CM2
ACCESS-ESM1-5
ACCESS-OM2
ACCESS-OM2-025
AWI-CM-1-1-HR
AWI-CM-1-1-LR
AWI-CM-1-1-MR
AWI-ESM-1-1-LR
BCC-CSM2-HR
BCC-CSM2-MR
BCC-ESM1
CAMS-CSM1-0
CAS-ESM2-0
CESM1-CAM5-SE-HR
CESM1-CAM5-SE-LR
CESM2
computing: /home/users/jmecking001/DATADIR/ACSIS/CMIP6/CESM2/ssp126/r4i1p1f1/zos_CESM2_ssp126_r4i1p1f1_gn_207001-209912_SC.nc
computing: /home/users/jmecking001/DATADIR/ACSIS/CMIP6/CESM2/ssp245/r4i1p1f1/zos_CESM2_ssp245_r4i1p1f1_gn_207001-209912_SC.nc
computing: /home/users/jmecking001/DATADIR/ACSIS/CMIP6/CESM2/ssp585/r4i1p1f1/zos_CESM2_ssp585_r4i1p1f1_gn_207001-209912_SC.nc
CESM2-FV2
CESM2-WACCM
computing: /home/users/jmecking001/DATADIR/ACSIS/CMIP6/CESM2-WACCM/ssp126/r1i1p1f1/zos_CESM2-WACCM_ssp126_r1i1p1f1_gn_207001-209912_SC.nc
computing: /home/users/jmecking001/DATADIR/ACSIS/CMIP6/CESM2-WACCM/ssp245/r1i1p1f1/zos_CESM2-WACCM_ssp245_r1i1p1f1_gn_207001-209912_SC.nc
computing: /home/users/jmecking001/DATADIR/ACSIS/CMIP6/CESM2-WACCM/ssp245/r2i1p1f1/zos_CESM2-WACCM_ssp245_r2i1p1f1_gn_20

KeyboardInterrupt: 